In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large")
input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Das Haus ist wunderbar.


In [11]:
text = """
Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned on a downstream task, has emerged as a powerful technique in natural language processing (NLP). The effectiveness of transfer learning has given rise to a diversity of approaches, methodology, and practice. In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework that converts every language problem into a text-to-text format. Our systematic study compares pre-training objectives, architectures, unlabeled datasets, transfer approaches, and other factors on dozens of language understanding tasks. By combining the insights from our exploration with scale and our new “Colossal Clean Crawled Corpus”, we achieve state-of-the-art results on many benchmarks covering summarization, question answering, text classification, and more. To facilitate future work on transfer learning for NLP, we release our dataset, pre-trained models, and code.
"""
input_ids = tokenizer("summarize:"+text, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

transfer learning is a technique where a model is first pre-trained on a data-rich task. it has emerged as a powerful technique in natural language processing (NLP) in this paper, we explore the landscape of transfer learning techniques for NLP.


In [14]:
import transformers
t5 = transformers.AutoModel.from_pretrained('t5-large')
print(t5.config.task_specific_params.keys())

dict_keys(['summarization', 'translation_en_to_de', 'translation_en_to_fr', 'translation_en_to_ro'])


In [15]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-small")

model = T5Model.from_pretrained("t5-small")

input_ids = tokenizer(

    "Studies have been shown that owning a dog is good for you", return_tensors="pt"

).input_ids  # Batch size 1

decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# preprocess: Prepend decoder_input_ids with start token which is pad token for T5Model.

# This is not needed for torch's T5ForConditionalGeneration as it does this internally using labels arg.

decoder_input_ids = model._shift_right(decoder_input_ids)

# forward pass

outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)

last_hidden_states = outputs.last_hidden_state

In [16]:
last_hidden_states

tensor([[[ 1.1526e-01,  1.1089e-01, -5.7172e-03,  ..., -4.8242e-03,
           4.1763e-04, -9.3368e-02],
         [ 8.7898e-02,  1.3796e-01, -3.2149e-02,  ...,  5.5419e-02,
           7.1102e-04, -6.8268e-03],
         [ 1.1366e-01,  6.3738e-02,  2.5220e-03,  ..., -4.4384e-02,
           3.6733e-05, -7.4255e-02],
         [ 2.0824e-01, -1.9027e-03,  1.0753e-01,  ..., -5.9003e-02,
           8.2739e-04,  4.5031e-03]]], grad_fn=<MulBackward0>)

In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")

model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids

labels = tokenizer("Das Haus ist wunderbar.", return_tensors="pt").input_ids

# the forward function automatically creates the correct decoder_input_ids

loss = model(input_ids=input_ids, labels=labels).loss

loss.item()

0.2542444169521332

In [18]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")

model = T5ForConditionalGeneration.from_pretrained("t5-small")

# training

input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids

labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

outputs = model(input_ids=input_ids, labels=labels)

loss = outputs.loss

logits = outputs.logits

# inference

input_ids = tokenizer(

    "summarize: studies have shown that owning a dog is good for you", return_tensors="pt"

).input_ids  # Batch size 1

outputs = model.generate(input_ids)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

studies have shown that owning a dog is good for you.


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")

model = T5ForConditionalGeneration.from_pretrained("t5-small")

# training

input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids

labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

outputs = model(input_ids=input_ids, labels=labels)

loss = outputs.loss

logits = outputs.logits

In [16]:
input_ids = tokenizer("I prefer <extra_id_0> to cats.", return_tensors="pt").input_ids
print(tokenizer.decode(input_ids[0], skip_special_tokens=False))
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

I prefer<extra_id_0> to cats.</s>
<pad> I prefer cats.</s>


In [17]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
input_ids = tokenizer.encode('summarize: I enjoy walking with my cute dog', return_tensors='pt')
greedy_output = model.generate(input_ids, num_beams=7, no_repeat_ngram_size=2, min_length=50, max_length=100)
print("Output:\n" + 100 * '-')

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog - he is a joy to walk with and is very affectionate with me. I like to spend time with him on walks with his kitty cat, leo, who is so cute!


In [18]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, I'm writing a new language for you. But first, I'd like to tell you about the language itself"},
 {'generated_text': "Hello, I'm a language model, and I'm trying to be as expressive as possible. In order to be expressive, it is necessary to know"},
 {'generated_text': "Hello, I'm a language model, so I don't get much of a license anymore, but I'm probably more familiar with other languages on that"},
 {'generated_text': "Hello, I'm a language model, a functional model... It's not me, it's me!\n\nI won't bore you with how"},
 {'generated_text': "Hello, I'm a language model, not an object model.\n\nIn a nutshell, I need to give language model a set of properties that"}]

In [39]:
import re
results = generator("'Great Expectations' was written by Charles Dickens. 'Hamlet' was written by", max_length=30, num_return_sequences=5)
for r in results:
    print(re.sub(r"\s+", " ", r['generated_text']))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Great Expectations' was written by Charles Dickens. 'Hamlet' was written by Charles Dickens. 'The Sun Is Set' was written by
'Great Expectations' was written by Charles Dickens. 'Hamlet' was written by Thomas C. Clarke. 'Das Chlorvish
'Great Expectations' was written by Charles Dickens. 'Hamlet' was written by J.R.R Tolkien. It was nominated
'Great Expectations' was written by Charles Dickens. 'Hamlet' was written by a different author. On the other hand, a number of
'Great Expectations' was written by Charles Dickens. 'Hamlet' was written by Charles Dickens. In the book's epilogue, he


In [37]:
import re
results = generator("'On the Origin of Species' was written by Charles Darwin. 'Das Kapital' was written by", max_length=30, num_return_sequences=5)
for r in results:
    print(re.sub(r"\s+", " ", r['generated_text']))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'On the Origin of Species' was written by Charles Darwin. 'Das Kapital' was written by William Lane Craig. 'De novo
'On the Origin of Species' was written by Charles Darwin. 'Das Kapital' was written by a French scientist, Jules Domen
'On the Origin of Species' was written by Charles Darwin. 'Das Kapital' was written by Christopher Columbus and Alfred New York. 
'On the Origin of Species' was written by Charles Darwin. 'Das Kapital' was written by Hermann-Bergner Geb
'On the Origin of Species' was written by Charles Darwin. 'Das Kapital' was written by Henry Holt. 'From the Great Man


In [26]:
generator("cheese = fromage, water =", max_length=30, num_return_sequences=5)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'cheese = fromage, water = from, gout = gout, k = k, m = m, s = s, t ='},
 {'generated_text': 'cheese = fromage, water = gold, milk = cow milk, eggs = milk, chicken = chicken breast, salmon = salmon fat = fish'},
 {'generated_text': 'cheese = fromage, water = water; [name] = family name; fromage and water have the same meaning as fromage and water'},
 {'generated_text': 'cheese = fromage, water = food, eggs = fromage, fish = fromage, etc. In one way or another, we can'},
 {'generated_text': 'cheese = fromage, water = old\n\n= old - fromage, water = old - 1 toage = fromage\n\n='}]

In [25]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [40]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='t5-large')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

ValueError: Could not load model t5-large with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>,).